## Librerías

In [6]:
# Librerías Personalizadas
import my_utils as ut

# Gestión de Archivos
import importlib

# Manejo de Data
import pandas as pd
import json

Este comando permite recargar las librerías que importamos sin necesidad de reiniciar el Kernel

In [18]:
importlib.reload(ut)

<module 'my_utils' from 'e:\\david\\Documents\\GitHub\\spotify-data-quality\\my_utils\\__init__.py'>

## Descarga de Archivos

La función importada descarga el archivo `taylor_swift_spotify.json` en el directorio `data/` si el archivo no se encuentra descargado en dicho directorio. Si el directorio `data/` no existe, lo crea. Esto lo hago como una buena práctica ya que los archivos JSON cuando provienen de APIs pueden ser de tamaños sumamente altos y, por lo tanto, es mejor usar un Script que los descargue directamente desde la fuente, ya sea la API o cualquier otro medio.

In [2]:
ut.descargarData()

Directorio "data/" creado
Descargando...
El archivo "taylor_swift_spotify.json" ha sido descargado en: "data/"


## Procesamiento de Datos

### Metodología de Trabajo

Con el fin de transcribir la información del archivo **JSON** a un archivo **.CSV** se creará un dataframe que contenga la información de cada nivel de anidación del archivo **JSON**. Cada dataframe contendrá a su vez una columna que le permita relacionarse con el nivel de anidación anterior para, finalmente, obtener un dataframe final con la totalidad de la información del archivo JSON desanidada.

Dada la estructura del JSON, es conveniente acoplar los 2 primeros niveles del JSON directamente en un dataframe. De esta forma, solo será necesario relacionar 2 dataframes:

* df_albums: Contiene la información del 1er y 2do nivel del JSON
* df_tracks: Contiene la información del 3er nivel del JSON desanidada

El dataframe final se exportará en formato **.CSV** cumpliendo así con la primera consigna.

### Carga del JSON a Diccionario

Para transformar y desanidar el archivo JSON apropiadamente es importante cargar los datos del archivo a un diccionario de Python. Para esto podemos abrir el archivo en modo lectura y, posteriormente, utilizar la función `json.load()` para almacenar el diccionario en la variable `data`

In [10]:
FILE_PATH = 'data/taylor_swift_spotify.json'

with open(FILE_PATH, 'r') as file:
    data = json.load(file)

### Creación del DataFrame 'Albums'

Para crear este DataFrame utilizamos la función `json.normalize()` de pandas, la cual permite obtener la información de una llave concreta del diccionario y almacenarla en un dataframe, así como normalizarla según sea necesario.

En este caso utilizaremos los siguientes parámetros para normalizar los 2 primeros niveles:

* data: Es la variable que contiene el diccionario con todos los datos del JSON
* record_path: Es la llave del diccionario que queremos desanidar
* meta: Son las llaves del diccionario que queremos preservar junto a la llave desanidada. En este caso serán todas las del primer nivel
* sep: El separador que indica las rutas de desanidado. Se establece con '.' ya que así se definió en el .CSV de ejemplo

In [58]:
df_albums = pd.json_normalize(data=data, 
                              record_path='albums', 
                              meta=['artist_id', 'artist_name', 'artist_popularity'],
                              sep='.'
                              )

df_albums.head(3)

,album_id,album_name,album_release_date,album_total_tracks,tracks,artist_id,artist_name,artist_popularity
0,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120
1,64LU4c1nfjz1t4VnGhagcg,1989 (Taylor's Version),2023-10-26,21,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120
2,5AEDGbliTTfjOB8TSm1sxt,Speak Now (Taylor's Version),2023-07-07,22,"[{'disc_number': 1, 'duration_ms': 231706, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120


El dataframe resultante contiene la columna **'tracks'**, la cual debemos eliminar puesto que la desanidaremos en otro dataframe.

In [59]:
df_albums.drop(columns='tracks', inplace=True)

df_albums.head(3)

,album_id,album_name,album_release_date,album_total_tracks,artist_id,artist_name,artist_popularity
0,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,06HL4z0CvFAxyc27GX,Taylor Swift,120
1,64LU4c1nfjz1t4VnGhagcg,1989 (Taylor's Version),2023-10-26,21,06HL4z0CvFAxyc27GX,Taylor Swift,120
2,5AEDGbliTTfjOB8TSm1sxt,Speak Now (Taylor's Version),2023-07-07,22,06HL4z0CvFAxyc27GX,Taylor Swift,120


### Creación del DataFrame 'Tracks'

La creación de este dataframe es muy similar al que se creó anteriormente. Sin embargo, ahora debemos especificar una ruta con la llave que queremos desanidar. Para ello, utilizamos el siguiente formato: `['llave', 'sub llave']`. La llave 'tracks' se encuentra dentro de la llave 'albums', así que lo especificamos.

De forma muy similar, en el parámetro 'meta' debemos especificar la ruta de la llave que queremos preservar para poder relacionar la información con el dataframe creado anteriormente. La sintaxis es: `[['llave', 'sub llave']]`. La llave 'album_id' se encuentra dentro de 'albums' así que lo especificamos.

In [45]:
df_tracks = pd.json_normalize(data=data, 
                              record_path=['albums', 'tracks'], 
                              meta=[['albums', 'album_id']], 
                              sep='.'
                              )

Ahora renombramos la columna **'albums.album_id'** a **'album_id'** tal como aparece en el **.CSV** de ejemplo.

In [47]:
df_tracks.rename(columns={'albums.album_id': 'album_id'}, inplace=True)
df_tracks.head(3)

,disc_number,duration_ms,explicit,track_number,track_popularity,track_id,track_name,audio_features.danceability,audio_features.energy,audio_features.key,...,audio_features.mode,audio_features.speechiness,audio_features.acousticness,audio_features.instrumentalness,audio_features.liveness,audio_features.valence,audio_features.tempo,audio_features.id,audio_features.time_signature,album_id
0,1,212600,False,1,77,4WUepByoeqcedHoYhSNHRt,Welcome To New York (Taylor's Version),0.757,0.610,7.0,...,1,0.0327,0.00942,0.000037,0.3670,0.685,116.998,4WUepByoeqcedHoYhSNHRt,4.0,1o59UpKw81iHR0HPiSkJR0
1,1,231833,False,2,78,0108kcWLnn2HlH2kedi1gn,Blank Space (Taylor's Version),0.733,0.733,0.0,...,1,0.0670,5,0,0.1680,0.701,96.057,0108kcWLnn2HlH2kedi1gn,4.0,1o59UpKw81iHR0HPiSkJR0
2,1,231000,False,3,79,3Vpk1hfMAQme8VJ0SNRSkd,Style (Taylor's Version),0.511,0.822,11.0,...,0,0.0397,0.000421,0.0197,0.0899,0.305,94.868,3Vpk1hfMAQme8VJ0SNRSkd,4.0,1o59UpKw81iHR0HPiSkJR0


### Creación del DataFrame final

Para obtener el dataframe final relacionamos la información de **'df_albums'** y **'df_tracks'** mediante la columna común **'album_id'**.

In [54]:
final_df = pd.merge(df_tracks, df_albums, on='album_id')
final_df.head(3)

,disc_number,duration_ms,explicit,track_number,track_popularity,track_id,track_name,audio_features.danceability,audio_features.energy,audio_features.key,...,audio_features.tempo,audio_features.id,audio_features.time_signature,album_id,album_name,album_release_date,album_total_tracks,artist_id,artist_name,artist_popularity
0,1,212600,False,1,77,4WUepByoeqcedHoYhSNHRt,Welcome To New York (Taylor's Version),0.757,0.610,7.0,...,116.998,4WUepByoeqcedHoYhSNHRt,4.0,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,06HL4z0CvFAxyc27GX,Taylor Swift,120
1,1,231833,False,2,78,0108kcWLnn2HlH2kedi1gn,Blank Space (Taylor's Version),0.733,0.733,0.0,...,96.057,0108kcWLnn2HlH2kedi1gn,4.0,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,06HL4z0CvFAxyc27GX,Taylor Swift,120
2,1,231000,False,3,79,3Vpk1hfMAQme8VJ0SNRSkd,Style (Taylor's Version),0.511,0.822,11.0,...,94.868,3Vpk1hfMAQme8VJ0SNRSkd,4.0,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,06HL4z0CvFAxyc27GX,Taylor Swift,120


Cabe aclarar que el orden de las columnas está ligeramente modificado de la siguiente forma:

* Actual: track_info -> album_info -> artist info
* Ejemplo: track_info -> artist_info -> album_info

Considero que el orden actual es un poco más legible ya que va de lo más específico a lo más general, así que el .CSV resultante tendrá ese mismo orden en las columnas

Finalmente exportamos el dataframe en formato **.CSV** y lo guardamos en el directorio `data/`

In [60]:
final_df.to_csv('data/taylor_swift_spotify.csv', index=False)